In [14]:
from tvb_multiscale.core.interfaces.transformers.models.elephant import *
import time

In [15]:
from quantities import Quantity, ms, s, sec, second, Hz, MHz

number_of_spiketrains = 1000
t_start = 0.0*ms
duration = 60000
t_stop = duration * 1.0*ms
sampling_period = 0.1*ms
rate_unit = Hz
time_unit = ms
as_array = True
refractory_period = 1.0*ms
rates = np.random.normal(loc=100.0, scale=10.0, size=(duration,1))

In [16]:
t = time.time()

spikes = []
for _ in range(10):

    spikes.append(compute_for_n_spiketrains(rates, number_of_spiketrains, refractory_period,
                              sampling_period, rate_unit, t_start, t_stop, as_array=True, analog_signal_class=None,
                              stationary_poisson_process=None, non_stationary_poisson_process=None))
    
print("%gms" % ((time.time()-t)*1e3))

2687.28ms


In [17]:
t = time.time()

refs = []
for _ in range(10):

    refs.append(ray_compute_for_n_spiketrains.remote(rates, number_of_spiketrains, refractory_period,
                                  sampling_period, rate_unit, t_start, t_stop, as_array=True, analog_signal_class=None))
    
outputs = ray.get(refs)

print("%gms" % ((time.time()-t)*1e3))

692.366ms


In [18]:
t = time.time()

refs = []
for _ in range(10):

    ray.get(ray_compute_for_n_spiketrains.remote(rates, number_of_spiketrains, refractory_period,
                                  sampling_period, rate_unit, t_start, t_stop, as_array=True, analog_signal_class=None))
    

print("%gms" % ((time.time()-t)*1e3))

2724.81ms


In [19]:
scale_factor = 1.0
translation_factor = 0.0
bin_size = sampling_period

In [20]:
for iP in range(10):
    sps = []
    for sp in spikes[iP]:
        sps += sp.tolist()
    spikes[iP] = np.array(sps)

In [21]:
t = time.time()

rates2 = []
for iP in range(10):

    rates2.append(compute_rates_ElephantSpikesHistogramRate(np.array(spikes[iP]),
                                                            bin_size, rate_unit, time_unit, t_start, t_stop, 
                                                            spike_train_class=None))
    
print("%gms" % ((time.time()-t)*1e3))

677.205ms


In [22]:
t = time.time()

refs = []
for iP in range(10):

    refs.append(ray_compute_rates_ElephantSpikesHistogramRate.remote(np.array(spikes[iP]), scale_factor, translation_factor,
                                                                     bin_size, rate_unit, time_unit, t_start, t_stop, spike_train_class=None))
    
outputs = ray.get(refs)
print("%gms" % ((time.time()-t)*1e3))

203.969ms


In [23]:
t = time.time()

outputs = []
for iP in range(10):

    outputs.append(ray.get(ray_compute_rates_ElephantSpikesHistogramRate.remote(np.array(spikes[iP]), scale_factor, translation_factor,
                                                                     bin_size, rate_unit, time_unit, t_start, t_stop, spike_train_class=None)))
    
print("%gms" % ((time.time()-t)*1e3))

813.548ms
